In [11]:
import numpy as np 
import pandas as pd
from fuzzywuzzy import fuzz
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from datetime import datetime
import joblib


In [2]:
df = pd.read_csv('C:/Users/ASUS/Desktop/Games/games.csv',encoding='latin1')

In [ ]:
df

,game_id,name,description,genres,platforms,rating,released,cover_image,game_link
0,3498,Grand Theft Auto V,"<p>Rockstar Games went bigger, since their pre...",Action,"PC, PlayStation 5, Xbox Series S/X, PlayStatio...",4.47,17-09-2013,https://media.rawg.io/media/games/20a/20aa03a1...,https://rawg.io/games/grand-theft-auto-v
1,3328,The Witcher 3: Wild Hunt,"<p>The third game in a series, it holds nothin...","Action, RPG","Xbox Series S/X, PlayStation 5, macOS, PlaySta...",4.65,18-05-2015,https://media.rawg.io/media/games/618/618c2031...,https://rawg.io/games/the-witcher-3-wild-hunt
2,4200,Portal 2,<p>Portal 2 is a first-person puzzle game deve...,"Shooter, Puzzle","PlayStation 3, PC, Xbox 360, Linux, macOS, Xbo...",4.59,18-04-2011,https://media.rawg.io/media/games/2ba/2bac0e87...,https://rawg.io/games/portal-2
3,4291,Counter-Strike: Global Offensive,<p>Counter-Strike is a multiplayer phenomenon ...,Shooter,"PC, Linux, Xbox 360, PlayStation 3",3.56,21-08-2012,https://media.rawg.io/media/games/736/73619bd3...,https://rawg.io/games/counter-strike-global-of...
4,5286,Tomb Raider (2013),<p>A cinematic revival of the series in its ac...,Action,"PlayStation 3, Xbox 360, macOS, PC, Xbox One, ...",4.06,05-03-2013,https://media.rawg.io/media/games/021/021c4e21...,https://rawg.io/games/tomb-raider
...,...,...,...,...,...,...,...,...,...
10404,5002,Cricket 07,Cricket 07 is a 2006 cricket simulation comput...,Sports,"PC,PlayStation,Windows",4.7,16-11-2006,https://images.launchbox-app.com/d6fccb94-c73b...,https://oldgamesdownload.com/cricket-07-bdt/
10405,5003,Cricket 24,Cricket 24 is a cricket video game developed b...,"Sports Video Game, Simulation Video Game, Simu...","PlaysStation,Xbox,Windows,PC,Nintendo Switch",3.5,05-10-2023,https://shared.cloudflare.steamstatic.com/stor...,https://store.steampowered.com/app/2358260/Cri...
10406,5004,Don Bradman Cricket 17,Don Bradman Cricket 17 is a cricket video game...,"Simulation Video Game, Sports","PlayStation 4, Xbox One, Microsoft Windows",3.45,15-12-2016,https://m.media-amazon.com/images/I/81vula5GKp...,https://store.steampowered.com/app/464850/Don_...
10407,5005,Ashes Cricket 2009,Ashes Cricket 2009 is a cricket video game dev...,"Sports Video Game, Simulation Game","Windows,Xbox 360,PlayStation",3.1,06-08-2009,https://m.media-amazon.com/images/I/81rZ6uv0gB...,https://www.cricketweb.net/game/ashes-cricket-...


In [4]:
df.fillna('', inplace=True)

In [5]:

# Validate required columns
required_columns = ["name", "genres", "platforms", "rating", "released", "cover_image", "game_link"]
missing_cols = [col for col in required_columns if col not in df.columns]
if missing_cols:
    raise KeyError(f"Missing columns: {missing_cols}. Available columns: {list(df.columns)}")

In [6]:
# Preprocess released to extract year
def extract_year(date_str):
    try:
        return datetime.strptime(date_str, "%d-%m-%Y").year 
    except:
        return None

In [7]:
df["release_year"] = df["released"].apply(extract_year)

# Preprocess: Combine features for content-based filtering
df["combined"] = df["genres"] + " " + df["platforms"]


In [8]:

# Vectorize features
vectorizer = TfidfVectorizer(stop_words="english")
feature_matrix = vectorizer.fit_transform(df["combined"])
similarity_matrix = cosine_similarity(feature_matrix)

In [ ]:

# Fuzzy matching function
def find_closest_game(query, titles):
    scores = [(title, fuzz.token_sort_ratio(query.lower(), title.lower())) for title in titles]
    best_match, score = max(scores, key=lambda x: x[1])
    if score < 50: 
        raise ValueError(f"No close match found for '{query}'")
    return best_match
def recommend_games(query, titles, similarity_matrix, data, top_n=5, date_weight=0.4):
    closest_title = find_closest_game(query, titles)
    game_idx = data[data["name"] == closest_title].index[0]
    matched_year = data.loc[game_idx, "release_year"]
    if matched_year is None:
        raise ValueError(f"Release year missing for '{closest_title}'")

    # Calculate combined scores: similarity + release date boost
    sim_scores = []
    for idx, sim in enumerate(similarity_matrix[game_idx]):
        if idx == game_idx:
            continue  # Skip the matched game itself
        release_year = data.loc[idx, "release_year"]
        # Boost games released after or in the same year
        year_score = 1.0 if release_year is not None and release_year >= matched_year else 0.0
        combined_score = (1 - date_weight) * sim + date_weight * year_score
        sim_scores.append((idx, combined_score))

    # Sort by combined score
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    top_indices = [idx for idx, score in sim_scores[:top_n]]

    # Return selected columns
    return data.iloc[top_indices][["name", "genres", "platforms", "rating",
                                   "released", "cover_image", "game_link"]]

In [10]:
recommend_games("elder ring", df["name"], similarity_matrix, df, top_n=5)

,name,genres,platforms,rating,released,cover_image,game_link
1159,Vampire: The Masquerade - Bloodlines 2,"Action, RPG","PC, PlayStation 5, Xbox Series S/X, Xbox One, ...",3.88,30-06-2025,https://media.rawg.io/media/games/fb5/fb5e0fdb...,https://rawg.io/games/vampire-the-masquerade-b...
2093,Diablo IV,"Action, RPG","PC, Xbox Series S/X, PlayStation 5, Xbox One, ...",3.79,06-06-2023,https://media.rawg.io/media/games/77d/77d51f8f...,https://rawg.io/games/diablo-iv
10314,ELEX II,"Action, RPG","PlayStation 4, Xbox One, Xbox Series S/X, Play...",3.34,01-03-2022,https://media.rawg.io/media/games/7bb/7bbc774e...,https://rawg.io/games/elex-ii
9208,Evil West,"Action, Adventure, RPG","PC, PlayStation 5, Xbox One, PlayStation 4, Xb...",3.65,21-11-2022,https://media.rawg.io/media/games/1b5/1b5182c7...,https://rawg.io/games/evil-west
10095,The First Descendant,"Action, Adventure, RPG","Xbox Series S/X, PlayStation 4, Xbox One, PC, ...",2.98,30-06-2024,https://media.rawg.io/media/screenshots/7a7/7a...,https://rawg.io/games/the-first-descendant


In [12]:
joblib.dump(vectorizer, 'tfidf_vectorizer.pkl')
joblib.dump(similarity_matrix, 'similarity_matrix.pkl')

['similarity_matrix.pkl']